In [ ]:
import sys
import os
import subprocess
from string import Template
import shutil

NUM_WORKERS = 3
os.environ['PDSH_SSH_ARGS_APPEND'] = ''
os.environ['NCCL_IB_DISABLE'] = '1'
os.environ['NCCL_IBEXT_DISABLE'] = '1'
os.environ['NCCL_DEBUG'] = 'INFO'
      
MASTER_IP = os.environ["CDSW_IP_ADDRESS"]
train_script="ft-LoRA-merge.py"
myhostfile="hostfile.txt"
deepspeed_cfg="dsconfig/zero3_fp16.json"
worker_gpu=1  
sshd_port=2222

deepspeed_cmd = "deepspeed --hostfile $myhostfile \
--launcher pdsh \
--num_gpus $worker_gpu --num_nodes $NUM_WORKERS \
--master_addr $MASTER_IP \
--ssh_port $sshd_port $train_script \
--deepspeed_config $deepspeed_cfg"

cml_cmd=Template(deepspeed_cmd).substitute(myhostfile=myhostfile, train_script=train_script, worker_gpu=worker_gpu, NUM_WORKERS=NUM_WORKERS, sshd_port=sshd_port, MASTER_IP=MASTER_IP, deepspeed_cfg=deepspeed_cfg)

In [ ]:
def remove_dir(dir_path):
    try:
        shutil.rmtree(dir_path)
        print(f"Folder '{dir_path}' has been deleted.")
    except Exception as e:
        # Ignore errors, you can print a message if needed
        print(f"Folder '{dir_path}' has been deleted.")
        
base_model = "bloom-1b1"
base_model_name = "bloom-1b1"
merged_model = "merged_bloom-1b1"
training_output = "training_bloom-1b1"
remove_dir(training_output) 
remove_dir(merged_model)

In [ ]:
main_cmd = subprocess.Popen([f'bash -c "{cml_cmd}" '], shell=True)
main_cmd.communicate()

In [ ]:
# Please restart the iPython kernel manually.